# Gemma-2b NSMC Finetuning

## 0. 미션
참조
- 정보: https://cloud.google.com/vertex-ai/docs/generative-ai/open-models/use-gemma?hl=ko
- 2b: https://huggingface.co/google/gemma-2b
- 2b instruction tuning: https://huggingface.co/google/gemma-1.1-2b-it
- 7b: https://huggingface.co/google/gemma-7b
- 7b instruction tuning: https://huggingface.co/google/gemma-1.1-7b-it
- nsmc: https://github.com/e9t/nsmc

미션
* 구글에서 공개한 gemma-2b-it를 활용하여 문장의 감정을 긍정/부정으로 예측하는 감정분류 데이터셋인 nmsc를 fine-tuning을 해 보면서 LLM 동작 방법과 fine-tuning 방법을 학습한다.

## 1. 라이브러리 설치 (최초 한번만 실행)
- 라이브러리는 colab이 최초 실행 또는 종료 후 실행된 경우 한번만 실행하면 됩니다.
- GPU 메모리 부족등의 이유로 colab 세션을 다시 시작한 경우는 설치할 필요 없습니다.
- colab 세션을 다시 시작하려면 '런타임' >> '세션 다시 시작'을 선택하세요.

In [ ]:
!pip install -q -U transformers==4.38.2
!pip install -q -U datasets==2.18.0
!pip install -q -U bitsandbytes==0.42.0
!pip install -q -U peft==0.9.0
!pip install -q -U trl==0.7.11
!pip install -q -U accelerate==0.27.2

## 2. 구글 드라이브 연결 (최초 한번만 실행)
- 구글 드라이브는 데이터 저장 및 학습 결과를 저장하기 위해서 사용합니다.
- 구글 드라이브는 colab이 최초 실행 또는 종료 후 실행된 경우 한번 만 연결하면 됩니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## *3. 환경 (매번 필수 실행)
- 환경은 colab 세션을 처음 시작하거나 다시 시작한 경우 실행되어야 합니다.
- 프로젝트 진행에 필요한 환경을 설정합니다.

### 3.1. 라이브러리 Import

In [ ]:
import os
import pandas as pd
from tqdm.auto import tqdm

import torch
from datasets import Dataset
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline,
                          TrainingArguments)
from peft import (LoraConfig,
                  PeftModel)
from trl import SFTTrainer

### 3.2. HuggingFace login
- 이번 프로젝트는 HuggingFace 로그인 해야만 진행이 가능합니다.
- HuggingFace 계정이 없다면 아래 URL에 접속해서 가입하시기 바랍니다.
  - https://huggingface.co/
- HuggingFace 로그인을 위해서 아래 URL에 접속해서 'User Access Token'을 생성하고 복사해서 Token에 입력하세요.
  - https://huggingface.co/settings/tokens

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# access token을 복사하세요.
HF_TOKEN = ""

### 3.3. 환경정보 설정
- WORKSPACE
  - 학습 데이터 및 학습결과를 저장하기 위한 경로입니다.
  - 필요할 경우 적당한 경로로 변경할 수 있습니다.
  - 경로를 변경 할 경우 전체 경로에 공백이 포함되지 않도록 주의해 주세요.
- MODEL_ID
  - 이번 프로젝트를 위한 LLM 입니다.
  - 구글에서 공개한 gemma-2b를 Instruction tunned한 버전입니다.
  - https://huggingface.co/google/gemma-2b-it

In [ ]:
WORKSPACE = '/content/drive/MyDrive/nlp-project'
MODEL_ID = 'google/gemma-1.1-2b-it'

## 4. Gemma understanding (재시작 필요)
- Gemma의 동작 및 사용 방법을 이해하기 위한 과정입니다.
- 이 과정을 시작하기 전 colab 세션을 다시 시작하세요.
- colab 세션을 다시 시작해야 하는 이유는 LLM의 model의 크기가 너무 크기 때문에 GPU의 메모리를 초기화 하기 위해서 입니다.

### 4.1. model load with 4 bits
- 2B token을 가진 gemma를 그냥 로딩할 경우는 약 9G의 GPU vRAM이 필요합니다.
- 4bit 양자화를 할 경우 2.2G의 GPU vRAM 필요.

In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
# load 4 bits model
model = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                                             device_map='auto',
                                             quantization_config=quantization_config,
                                             token=HF_TOKEN)
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID,
                                          add_special_tokens=True,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

### 4.2. pipeline
- https://huggingface.co/docs/transformers/main_classes/pipelines
- huggingface에서 inference를 쉽게 하기 위해 정의한 라이브러리.

In [ ]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512)
pipe

### 4.3. gemma prompt
- https://huggingface.co/google/gemma-1.1-2b-it
- 아래와 같은 형식이 gemma의 prompt 형식 입니다.
```
<bos><start_of_turn>user
{content}<end_of_turn>
<start_of_turn>model
```
- NSMC 추론을 위한 프롬프트를 생성하는 과정입니다.

In [ ]:
doc = """엄청나게 즐거운 시간이었습니다. 강추!!!"""

In [ ]:
messages = [
    {
        "role": "user",
        "content": "다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:\n\n{}".format(doc)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages,
                                            tokenize=False,
                                            add_generation_prompt=True)

In [ ]:
print(prompt)

### 4.4. gemma inference
- 이전 단계에서 생성한 prompt를 이용해 추론하고 결과를 확인하는 과장입니다.

In [ ]:
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
outputs

In [ ]:
print(outputs[0]["generated_text"])

In [ ]:
print(outputs[0]["generated_text"][len(prompt):])

### 4.5. gemma chat
- chatbot 형식의 감정분류 예 입니다.

In [ ]:
def gen_prompt(pipe, doc):
    messages = [
        {
            "role": "user",
            "content": "다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:\n\n{}".format(doc)
        }
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages,
                                                tokenize=False,
                                                add_generation_prompt=True)
    return prompt

In [ ]:
def gen_response(pipe, doc):
    prompt = gen_prompt(pipe, doc)

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        add_special_tokens=True
    )
    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
while True:
    doc = input('문장 > ')
    doc = doc.strip()
    if len(doc) == 0:
        break
    result = gen_response(pipe, doc)
    print(f'감정 > {result}\n\n')

## 5. Gemma-2b 학습 tutorial (재시작 필요)
- Gemma의 학습과정을 이해하기 위한 과정입니다.
- 이 과정을 시작하기 전 colab 세션을 다시 시작하세요.
- colab 세션을 다시 시작해야 하는 이유는 LLM의 model의 크기가 너무 크기 때문에 GPU의 메모리를 초기화 하기 위해서 입니다.

### 5.1. Gemma를 로딩 (4bit)

In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # NormalFloat 4
    bnb_4bit_compute_dtype=torch.float16
)
# load 4 bits model
model = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                                             device_map='auto',
                                             quantization_config=quantization_config,
                                             token=HF_TOKEN)
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID,
                                          add_special_tokens=True,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

### 5.2. LoRA를 적용하기 위한 설정을 정의

In [ ]:
# lora config
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

### 5.3. Dataset 정의
- 동작 확인을 위해서 최소한의 데이터만을 사용합니다.

In [ ]:
# huggingface dataset을 생성하는 함수입니다.
def make_dataset(df, sample=-1):
    df = df[['document', 'label']]
    if sample > 0:
        df = df.sample(sample)
    dataset = Dataset.from_pandas(df)
    return dataset

In [ ]:
data = {
    'document': [
        '영화 강추 합니다.',
        '시간이 너무 아깝습니다.'
    ],
    'label': [
        1,
        0
    ]
}
df_train = pd.DataFrame(data)
df_train

In [ ]:
train_dataset = make_dataset(df_train)
train_dataset

### 5.4. 학습을 위한 prompt 생성 함수 정의

In [ ]:
# 학습을 위한 prompt를 생성합니다.
def gen_train_prompt(example):
    prompt_list = []
    for i in range(len(example['document'])):
        doc = example['document'][i]
        label = '긍정' if example['label'][i] == 1 else '부정'
        prompt_list.append(r"""<bos><start_of_turn>user
다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:

{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(doc, label))
    return prompt_list

In [ ]:
# prompt 동작을 확인합니다.
prompt = gen_train_prompt(train_dataset[:1])[0]
print(prompt)

### 5.5. Trainer를 이용한 학습
- https://huggingface.co/docs/trl/sft_trainer
- https://huggingface.co/docs/trl/v0.8.4/en/sft_trainer#trl.SFTTrainer
- https://huggingface.co/docs/transformers/v4.39.3/en/main_classes/trainer#transformers.TrainingArguments

In [ ]:
# trainer 정의
trainer = SFTTrainer(
    model=model, # 학습할 모델
    train_dataset=train_dataset,  # 학습할 데이터 셋
    max_seq_length=256,  # 최대 토큰 갯수
    args=TrainingArguments(
        output_dir=os.path.join(WORKSPACE, "nsmc-tutorial"),
        # num_train_epochs = 1,  # epoc으로 할 경우 너무 많이 걸리 수 있음
        max_steps=10,  # 학습 step 수
        per_device_train_batch_size=2,  # gpu당 입력 batch_size
        gradient_accumulation_steps=4,  # gradient 누적 후 학습
        optim="paged_adamw_8bit",  # optimizer (QLoRA)
        warmup_steps=1000,  # learning rate warmup step
        learning_rate=1e-4,  # learning rate
        # bf16=True,  # bf16 사용 여부 (3090 이상에서 가능)
        fp16=True,  # fp16 사용 여부 (예전 GPU에서 사용 가능, T4)
        logging_steps=10,  # 얼마만에 한번 씩 중간 결과를 확인할 것인가?
        push_to_hub=False,  # huggingface에 올릴 수 있음
        report_to=None,  # W&B에 학습결과 공유 가능
    ),
    peft_config=lora_config,  # QLoRA config
    formatting_func=gen_train_prompt,  # 프롬프트 생성 함수
)

In [ ]:
# train
trainer.train()

### 5.6. LoRA의 학습된 weight 저장
- 전체 weight가 아닌 변화량만 저장하므로 매우 작은 용량을 차지합니다.

In [ ]:
type(trainer.model)

In [ ]:
# save lora (delta weight)
trainer.model.save_pretrained("lora_adapter")

In [ ]:
# 저장된 결과 확인
!ls -lh ./lora_adapter

### 5.7. Original 모델에 LoRA를 학습된 결과를 더해서 최종 모델 저장

In [ ]:
# original model load (before finetuned)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             token=HF_TOKEN)

In [ ]:
# merge : original + delta wieght
model = PeftModel.from_pretrained(model,
                                  "lora_adapter",
                                  device_map='auto',
                                  torch_dtype=torch.float16,
                                  token=HF_TOKEN)
model = model.merge_and_unload()

In [ ]:
# save fine-tunned model
model.save_pretrained(os.path.join("nsmc-tutorial", "checkpoint-final"))

In [ ]:
# 저장 결과 확인
!ls -lh nsmc-tutorial/checkpoint-final

### 5.8. 학습 결과 평가

In [ ]:
# 평가 데이터
data = {
    'document': [
        '영화 재밌어요',
        '졸려서 눈물이 났어요.'
    ],
    'label': [
        1,
        0
    ]
}
df_test = pd.DataFrame(data)
df_test

In [ ]:
# 평가 데이터셋 생성
test_dataset = make_dataset(df_test)
test_dataset

In [ ]:
# 평가를 위한 prompt
def gen_test_prompt(example):
    prompt_list = []
    for i in range(len(example['document'])):
        doc = example['document'][i]
        prompt_list.append(r"""<bos><start_of_turn>user
다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:

{}<end_of_turn>
<start_of_turn>model
""".format(doc))
    return prompt_list

In [ ]:
# prompt 확인
prompt = gen_test_prompt(test_dataset[1:])[0]
print(prompt)

In [ ]:
# pipeline 정의
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=10)

In [ ]:
# infer
total_sample_cnt, total_correct_cnt = 0, 0
for example in test_dataset.iter(1):
    label = '긍정' if example['label'][0] == 1 else '부정'

    prompt = gen_test_prompt(example)
    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        add_special_tokens=True
    )
    pred = outputs[0][0]['generated_text'][len(prompt[0]):]
    total_sample_cnt += 1
    total_correct_cnt += 1 if label == pred else 0

    print(example['document'][0], ":", pred)
    print('-' * 20)
print(f"Test Accuracy: {total_correct_cnt} / {total_sample_cnt} = {total_correct_cnt/total_sample_cnt:.4f}")

## 6. Gemma-2b 학습 실습 (재시작 필요)
- '5. Gemma-2b 학습 tutorial'를 참고해서 실제로 Gemma-2b를 학습해 보는 과정입니다.
- 이 과정을 시작하기 전 colab 세션을 다시 시작하세요.
- colab 세션을 다시 시작해야 하는 이유는 LLM의 model의 크기가 너무 크기 때문에 GPU의 메모리를 초기화 하기 위해서 입니다.

### 6.1 학습 및 테스트 데이터를 다운로드
- https://github.com/e9t/nsmc

In [ ]:
# 데이터를 다운로드할 폴더를 생성합니다.
os.makedirs(os.path.join(WORKSPACE, "data", "nsmc"), exist_ok=True)

In [ ]:
!wget https://github.com/e9t/nsmc/raw/master/ratings_train.txt \
    -O {WORKSPACE}/data/nsmc/train.tsv

In [ ]:
!wget https://github.com/e9t/nsmc/raw/master/ratings_test.txt \
    -O {WORKSPACE}/data/nsmc/test.tsv

In [ ]:
!ls {WORKSPACE}/data/nsmc

In [ ]:
df_train = pd.read_csv(os.path.join(WORKSPACE, "data", "nsmc", "train.tsv"),
                       sep='\t')
df_train

In [ ]:
df_test = pd.read_csv(os.path.join(WORKSPACE, "data", "nsmc", "test.tsv"),
                      sep='\t')
df_test

### 6.2. Gemma를 로딩 (4bit)

In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
# load 4 bits model
model = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                                             device_map='auto',
                                             quantization_config=quantization_config,
                                             token=HF_TOKEN)
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID,
                                          add_special_tokens=True,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

### 6.3. LoRA를 적용하기 위한 설정 정의

In [ ]:
# lora config
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

### 6.4. Train Dataset을 생성
- 이전에 다운 받은 {WORKSPACE}/data/nsmc/train.tsv 파일을 학습 데이터로 사용합니다.
- 학습 데이터는 df_train으로 이미 읽어진 상태입니다.

In [ ]:
# huggingface dataset을 생성하는 함수입니다.
def make_dataset(df, sample=-1):
    df = df[['document', 'label']]
    if sample > 0:
        df = df.sample(sample)
    dataset = Dataset.from_pandas(df)
    return dataset

In [ ]:
# 15만개의 학습 데이터셋을 생성합니다.
train_dataset = make_dataset(df_train)
train_dataset

### 6.5. 학습을 위한 prompt 생성 함수를 정의합니다.

In [ ]:
# 학습을 위한 prompt를 생성합니다.
def gen_train_prompt(example):
    prompt_list = []
    for i in range(len(example['document'])):
        doc = example['document'][i]
        label = '긍정' if example['label'][i] == 1 else '부정'
        prompt_list.append(r"""<bos><start_of_turn>user
다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:

{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(doc, label))
    return prompt_list

In [ ]:
# prompt 동작을 확인합니다.
prompt = gen_train_prompt(train_dataset[:1])[0]
print(prompt)

### 6.6. Trainer를 이용한 학습
- https://huggingface.co/docs/trl/sft_trainer
- https://huggingface.co/docs/trl/v0.8.4/en/sft_trainer#trl.SFTTrainer
- https://huggingface.co/docs/transformers/v4.39.3/en/main_classes/trainer#transformers.TrainingArguments
- tutorial의 설정 중 다음 내용만 변경하셔도 됩니다.
  - output_dir: {WORKSPACE}/nsmc-gemma
  - max_steps: 1000 ~ 2000 사이
  - logging_steps: 100

In [ ]:
# trainer 정의
trainer = SFTTrainer(
    model=model, # 학습할 모델
    train_dataset=train_dataset,  # 학습할 데이터 셋
    max_seq_length=256,  # 최대 토큰 갯수
    args=TrainingArguments(
        output_dir=os.path.join(WORKSPACE, "nsmc-gemma"),
        # num_train_epochs = 1,  # epoc으로 할 경우 너무 많이 걸리 수 있음
        max_steps=1000,  # 학습 step 수 (1000 ~ 2000 사이)
        per_device_train_batch_size=2,  # gpu당 입력 batch_size
        gradient_accumulation_steps=4,  # gradient 누적 후 학습
        optim="paged_adamw_8bit",  # optimizer (QLoRA)
        warmup_steps=1000,  # learning rate warmup step
        learning_rate=1e-4,  # learning rate
        # bf16=True,  # bf16 사용 여부 (3090 이상에서 가능)
        fp16=True,  # fp16 사용 여부 (예전 GPU에서 사용 가능, T4)
        logging_steps=100,  # 얼마만에 한번 씩 중간 결과를 확인할 것인가?
        push_to_hub=False,  # hugingface에 올릴 수 있음
        report_to=None,  # W&B에 학습결과 공유 가능
    ),
    peft_config=lora_config,  # QLoRA config
    formatting_func=gen_train_prompt,  # 프롬프트 생성 함수
)

In [ ]:
# train
trainer.train()

### 6.7. LoRA의 학습된 weight 저장
- 전체 weight가 아닌 변화량만 저장하므로 매우 작은 용량을 차지합니다.

In [ ]:
# save lora (delta weight)
trainer.model.save_pretrained("lora_adapter")

In [ ]:
# 저장된 결과 확인
!ls -lh ./lora_adapter

### 6.8. Original 모델에 LoRA를 학습된 결과를 더해서 최종 모델 저장.
- 최종 모델 저장 위치: {WORKSPACE}/nsmc-gemma/checkpoint-final

In [ ]:
# original model load (before finetuned)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             token=HF_TOKEN)

In [ ]:
# merge : original + delta wieght
model = PeftModel.from_pretrained(model,
                                  "lora_adapter",
                                  device_map='auto',
                                  torch_dtype=torch.float16,
                                  token=HF_TOKEN)
model = model.merge_and_unload()

In [ ]:
# save fine-tunned model
model.save_pretrained(os.path.join(WORKSPACE, "nsmc-gemma", "checkpoint-final"))

In [ ]:
# 저장 결과 확인
!ls -lh {WORKSPACE}/nsmc-gemma/checkpoint-final

## 7. Gemma-2b 평가 및 추론 실습 (재시작 필요)
- '6. Gemma-2b 학습 실습'을 완료한 후 결과를 평가해 보는 과정입니다.
- '5. Gemma-2b 학습 tutorial'의 '5.8. 학습결과 평가' 섹션을 참고하세요.
- 이 과정을 시작하기 전 colab 세션을 다시 시작하세요.
- colab 세션을 다시 시작해야 하는 이유는 LLM의 model의 크기가 너무 크기 때문에 GPU의 메모리를 초기화 하기 위해서 입니다.

### 7.1. Test Data loading
- 평가를 위한 테스트 데이터를 로딩합니다.

In [ ]:
df_test = pd.read_csv(os.path.join(WORKSPACE, "data", "nsmc", "test.tsv"),
                      sep='\t')
df_test

### 7.2. 학습된 Gemma를 로딩 (4bit)
- AutoModelForCausalLM의 MODEL_ID를 '{WORKSPACE}/nsmc-gemma/checkpoint-final'로 변경합니다.
- 위 경로가 우리가 학습한 모델이 저장된 위치입니다.
- AutoTokenizer는 MODEL_ID를 그대로 사용합니다.

In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
# load 4 bits model
model_fn = os.path.join(WORKSPACE, "nsmc-gemma", "checkpoint-final")
model = AutoModelForCausalLM.from_pretrained(model_fn,
                                             device_map='auto',
                                             quantization_config=quantization_config)
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID,
                                          add_special_tokens=True)
tokenizer.padding_side = 'right'

### 7.3. 학습 결과 평가를 위한 데이터 준비

In [ ]:
# huggingface dataset을 생성하는 함수입니다.
def make_dataset(df, sample=-1):
    df = df[['document', 'label']]
    if sample > 0:
        df = df.sample(sample)
    dataset = Dataset.from_pandas(df)
    return dataset

In [ ]:
# 평가 데이터셋 새성
test_dataset = make_dataset(df_test)
test_dataset

### 7.4. 테스트를 위한 프롬프트 생성 및 확인

In [ ]:
# 평가를 위한 prompt
def gen_test_prompt(example):
    prompt_list = []
    for i in range(len(example['document'])):
        doc = example['document'][i]
        prompt_list.append(r"""<bos><start_of_turn>user
다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:

{}<end_of_turn>
<start_of_turn>model
""".format(doc))
    return prompt_list

In [ ]:
# prompt 확인
prompt = gen_test_prompt(test_dataset[1:])[0]
print(prompt)

### 7.5. 파이프라인 정의 및 평가
- 평가는 1000개만 실행하세요. (시간이 오래 걸립니다.)

In [ ]:
# pipeline 정의
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=10)

In [ ]:
# infer
total_sample_cnt, total_correct_cnt = 0, 0
for example in tqdm(test_dataset.iter(1)):
    label = '긍정' if example['label'][0] == 1 else '부정'

    prompt = gen_test_prompt(example)
    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        add_special_tokens=True
    )
    pred = outputs[0][0]['generated_text'][len(prompt[0]):]
    total_sample_cnt += 1
    total_correct_cnt += 1 if label == pred else 0

    # print(example['document'][0], ":", pred)
    # print('-' * 20)

    if total_sample_cnt >= 1000:
        break
print(f"Test Accuracy: {total_correct_cnt} / {total_sample_cnt} = {total_correct_cnt/total_sample_cnt:.4f}")

### 7.6. gemma chat
- chatbot 형식의 감정분류 입니다.
- '4.5. gemma chat' 섹션을 참고하세요.

In [ ]:
def gen_prompt(pipe, doc):
    messages = [
        {
            "role": "user",
            "content": "다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:\n\n{}".format(doc)
        }
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages,
                                                tokenize=False,
                                                add_generation_prompt=True)
    return prompt

In [ ]:
def gen_response(pipe, doc):
    prompt = gen_prompt(pipe, doc)

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        add_special_tokens=True
    )
    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
while True:
    doc = input('문장 > ')
    doc = doc.strip()
    if len(doc) == 0:
        break
    result = gen_response(pipe, doc)
    print(f'감정 > {result}\n\n')